In [1]:
import json
import boto3
import pyspark
import findspark
import multiprocessing
findspark.init('/home/martin96/Spark/spark-3.2.1-bin-hadoop3.2')
s3_client = boto3.client('s3')
sessions3 = boto3.Session()
s3 = sessions3.resource('s3')
my_bucket = s3.Bucket('pinterestkafkabucket')

conf = (
    pyspark.SparkConf()
    # Setting where master node is located [cores for multiprocessing]
    .setMaster(f"local[{multiprocessing.cpu_count()}]")
    # Setting application name
    .setAppName("TestApp")
    # Setting config value via string
    .set("spark.eventLog.enabled", False)
    # Setting environment variables for executors to use
    .setExecutorEnv(pairs=[("VAR3", "value3"), ("VAR4", "value4")])
    # Setting memory if this setting was not set previously
    .setIfMissing("spark.executor.memory", "1g")
)
sessionsp = pyspark.sql.SparkSession.builder.config(conf=conf).getOrCreate()
sc = sessionsp.sparkContext

22/03/24 16:19:07 WARN Utils: Your hostname, martin96-TP300LA resolves to a loopback address: 127.0.1.1; using 192.168.1.109 instead (on interface wlp2s0)
22/03/24 16:19:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/24 16:19:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pandas as pd
from functools import reduce
#df = sessionsp.createDataFrame(pkeys.collect())
df_list = []
for obj in my_bucket.objects.all():
### Read streaming body
    obj = s3_client.get_object(Bucket='pinterestkafkabucket', Key=obj.key)
    content = obj['Body'].read()
    data = json.loads(content)
    pkeys = sc.parallelize([data])
    norm = pd.json_normalize(pkeys.collect())
    norm_list = norm.iloc[0].tolist()
    df = sessionsp.createDataFrame(pd.DataFrame(norm, columns=norm.keys()))
    df_list.append(df)
df = reduce(pyspark.sql.DataFrame.union, df_list)
df.show()
#df.select("category").show()

22/03/24 16:21:10 WARN DAGScheduler: Broadcasting large task binary with size 1899.3 KiB
22/03/24 16:21:12 WARN DAGScheduler: Broadcasting large task binary with size 1899.3 KiB
22/03/24 16:21:13 WARN DAGScheduler: Broadcasting large task binary with size 1899.3 KiB
22/03/24 16:21:15 WARN DAGScheduler: Broadcasting large task binary with size 1899.3 KiB


+--------------+-----+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+----------+--------------------+
|      category|index|           unique_id|               title|         description| follower_count|            tag_list|   is_image_or_video|           image_src|downloaded|       save_location|
+--------------+-----+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+----------+--------------------+
|        travel|10020|56cac475-b169-42c...|10 Tips for Plann...|Ten tips to keep ...|             5k|Scotland Vacation...|               image|https://i.pinimg....|         1|Local save in /da...|
|           art|  365|ee28bec4-ac3b-458...|THE BEYOND T-Shir...|Lucio Fulci's hor...|             53|Creepy,Scary,Terr...|               image|https://i.pinimg....|         1|Local save in /da...|
|diy-and-crafts

In [13]:
sp_table_string = "index INTEGER :key,"

for col in df.columns:
    if col != "index":
        if col == "downloaded":
            sp_table_string += f"{col} INTEGER Incoming_Data:{col},"
        else:
            sp_table_string += f"{col} STRING Incoming_Data:{col},"

sp_table_string = sp_table_string[:(len(sp_table_string)-1)]
print(sp_table_string)

index STRING :key,category STRING cf:Incoming_Data,unique_id STRING cf:Incoming_Data,title STRING cf:Incoming_Data,description STRING cf:Incoming_Data,follower_count STRING cf:Incoming_Data,tag_list STRING cf:Incoming_Data,is_image_or_video STRING cf:Incoming_Data,image_src STRING cf:Incoming_Data,downloaded STRING cf:Incoming_Data,save_location STRING cf:Incoming_Data


In [11]:
# postgres table
pg_table_string = "ALTER TABLE pinterest_data ADD COLUMN "

for col in df.columns:
    if col != "index":
        if col == "downloaded":
            pg_table_string += f"{col} INTEGER, ADD COLUMN "
        else:
            pg_table_string += f"{col} VARCHAR, ADD COLUMN "

pg_table_string = pg_table_string[:(len(pg_table_string)-1)]
print(pg_table_string)

ALTER TABLE pinterest_data ADD COLUMN category VARCHAR, ADD COLUMN unique_id VARCHAR, ADD COLUMN title VARCHAR, ADD COLUMN description VARCHAR, ADD COLUMN follower_count VARCHAR, ADD COLUMN tag_list VARCHAR, ADD COLUMN is_image_or_video VARCHAR, ADD COLUMN image_src VARCHAR, ADD COLUMN downloaded INTEGER, ADD COLUMN save_location VARCHAR, ADD COLUMN


In [27]:
import findspark
import multiprocessing
from pyspark.sql import SparkSession
from pyspark import SQLContext, SparkContext, SparkConf

findspark.init('/home/martin96/Spark/spark-3.2.1-bin-hadoop3.2')
findspark.find()

sp_table_string = "index int :key,"
for col in df.columns:
    if col != "index":
        if col == "downloaded":
            sp_table_string += f"{col} int Incoming_Data:{col},"
        else:
            sp_table_string += f"{col} STRING Incoming_Data:{col},"

sp_table_string = sp_table_string[:(len(sp_table_string)-1)]
# print(sp_table_string)

def write_to_hbase():
    conf = (SparkConf().setAppName("RW_from_HBase"))

    spark = SparkSession.builder.appName(" ").config(conf=conf).getOrCreate()

    # hb_df = spark.read.format("org.apache.hadoop.hbase.spark").option("hbase.columns.mapping",
    #         f"{sp_table_string}").option("hbase.spark.pushdown.columnfilter", True) \
    #             .option("hbase.table", "Pinterest_Data_v2") \
    #             .option("hbase.spark.use.hbasecontext", False) \
    #             .load()

    df.write.format("org.apache.hadoop.hbase.spark").option("hbase.columns.mapping",
            f"{sp_table_string}").option("hbase.spark.pushdown.columnfilter", True) \
                .option("hbase.table", "Pinterest_Data_v2") \
                .option("hbase.spark.use.hbasecontext", False) \
                .save()
    df.createOrReplaceTempView("table")
    # #spark.sql("INSERT INTO table (rowKey, f_name, l_name) VALUES ('3', 'steve','smith')")
    
    df.show()

write_to_hbase()

index int :key,category STRING Incoming_Data:category,unique_id STRING Incoming_Data:unique_id,title STRING Incoming_Data:title,description STRING Incoming_Data:description,follower_count STRING Incoming_Data:follower_count,tag_list STRING Incoming_Data:tag_list,is_image_or_video STRING Incoming_Data:is_image_or_video,image_src STRING Incoming_Data:image_src,downloaded int Incoming_Data:downloaded,save_location STRING Incoming_Data:save_location


22/03/10 21:08:05 WARN FileOutputCommitter: Output Path is null in setupJob()
22/03/10 21:08:08 WARN ClientCnxn: An exception was thrown while closing send thread for session 0x17f75917dc90064.
EndOfStreamException: Unable to read additional data from server sessionid 0x17f75917dc90064, likely server has closed socket
	at org.apache.zookeeper.ClientCnxnSocketNIO.doIO(ClientCnxnSocketNIO.java:77)
	at org.apache.zookeeper.ClientCnxnSocketNIO.doTransport(ClientCnxnSocketNIO.java:350)
	at org.apache.zookeeper.ClientCnxn$SendThread.run(ClientCnxn.java:1275)
22/03/10 21:08:10 WARN ClientCnxn: An exception was thrown while closing send thread for session 0x17f75917dc90075.
EndOfStreamException: Unable to read additional data from server sessionid 0x17f75917dc90075, likely server has closed socket
	at org.apache.zookeeper.ClientCnxnSocketNIO.doIO(ClientCnxnSocketNIO.java:77)
	at org.apache.zookeeper.ClientCnxnSocketNIO.doTransport(ClientCnxnSocketNIO.java:350)
	at org.apache.zookeeper.ClientCn

+--------------+-----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+----------+--------------------+
|      category|index|           unique_id|               title|         description|follower_count|            tag_list|   is_image_or_video|           image_src|downloaded|       save_location|
+--------------+-----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+----------+--------------------+
|        travel|10020|56cac475-b169-42c...|10 Tips for Plann...|Ten tips to keep ...|            5k|Scotland Vacation...|               image|https://i.pinimg....|         1|Local save in /da...|
|           art|  365|ee28bec4-ac3b-458...|THE BEYOND T-Shir...|Lucio Fulci's hor...|            53|Creepy,Scary,Terr...|               image|https://i.pinimg....|         1|Local save in /da...|
|diy-and-crafts| 297

In [29]:
import findspark
import multiprocessing
from pyspark.sql import SparkSession
from pyspark import SQLContext, SparkContext, SparkConf

findspark.init('/home/martin96/Spark/spark-3.2.1-bin-hadoop3.2')
findspark.find()
spark = SparkSession.builder.appName(" ").config(conf=conf).getOrCreate()
hb_df = spark.read.format("org.apache.hadoop.hbase.spark").option("hbase.columns.mapping",
        f"{sp_table_string}").option("hbase.spark.pushdown.columnfilter", True) \
            .option("hbase.table", "Pinterest_Data_v2") \
            .option("hbase.spark.use.hbasecontext", False) \
            .load()
hb_df.select('index').show()

index int :key,category STRING Incoming_Data:category,unique_id STRING Incoming_Data:unique_id,title STRING Incoming_Data:title,description STRING Incoming_Data:description,follower_count STRING Incoming_Data:follower_count,tag_list STRING Incoming_Data:tag_list,is_image_or_video STRING Incoming_Data:is_image_or_video,image_src STRING Incoming_Data:image_src,downloaded int Incoming_Data:downloaded,save_location STRING Incoming_Data:save_location
+-----+
|index|
+-----+
|  365|
|  637|
| 1667|
| 2975|
| 3032|
| 3813|
| 4249|
| 7467|
|10020|
|10029|
|10538|
+-----+



In [31]:
hb_df.createOrReplaceTempView("table")
my_row = spark.sql("SELECT * FROM table WHERE index=365;")
my_row.show()

22/03/10 21:09:10 ERROR Executor: Exception in task 0.0 in stage 44.0 (TID 366)
org.apache.hadoop.hbase.DoNotRetryIOException: org.apache.hadoop.hbase.DoNotRetryIOException: java.lang.ClassNotFoundException: org.apache.hadoop.hbase.spark.SparkSQLPushDownFilter
	at org.apache.hadoop.hbase.protobuf.ProtobufUtil.toFilter(ProtobufUtil.java:1724)
	at org.apache.hadoop.hbase.protobuf.ProtobufUtil.toGet(ProtobufUtil.java:556)
	at org.apache.hadoop.hbase.regionserver.RSRpcServices.get(RSRpcServices.java:2363)
	at org.apache.hadoop.hbase.protobuf.generated.ClientProtos$ClientService$2.callBlockingMethod(ClientProtos.java:36800)
	at org.apache.hadoop.hbase.ipc.RpcServer.call(RpcServer.java:2423)
	at org.apache.hadoop.hbase.ipc.CallRunner.run(CallRunner.java:124)
	at org.apache.hadoop.hbase.ipc.RpcExecutor$Handler.run(RpcExecutor.java:311)
	at org.apache.hadoop.hbase.ipc.RpcExecutor$Handler.run(RpcExecutor.java:291)
Caused by: java.lang.ClassNotFoundException: org.apache.hadoop.hbase.spark.SparkS

Py4JJavaError: An error occurred while calling o722.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 44.0 failed 1 times, most recent failure: Lost task 0.0 in stage 44.0 (TID 366) (martin96-TP300LA.lan executor driver): org.apache.hadoop.hbase.DoNotRetryIOException: org.apache.hadoop.hbase.DoNotRetryIOException: java.lang.ClassNotFoundException: org.apache.hadoop.hbase.spark.SparkSQLPushDownFilter
	at org.apache.hadoop.hbase.protobuf.ProtobufUtil.toFilter(ProtobufUtil.java:1724)
	at org.apache.hadoop.hbase.protobuf.ProtobufUtil.toGet(ProtobufUtil.java:556)
	at org.apache.hadoop.hbase.regionserver.RSRpcServices.get(RSRpcServices.java:2363)
	at org.apache.hadoop.hbase.protobuf.generated.ClientProtos$ClientService$2.callBlockingMethod(ClientProtos.java:36800)
	at org.apache.hadoop.hbase.ipc.RpcServer.call(RpcServer.java:2423)
	at org.apache.hadoop.hbase.ipc.CallRunner.run(CallRunner.java:124)
	at org.apache.hadoop.hbase.ipc.RpcExecutor$Hand